<a href="https://colab.research.google.com/github/Keoni808/NFL_Data_Cleaning/blob/main/NFL_Plays_Week1_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PURPOSE:
- To view a larger sample size of plays.
  - Currently working on breaking down a single game but do not have enough data in that game to correctly break down all play descriptions for different play types.

# MOUNTING AND IMPORTS

In [ ]:
# Mount your Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Used to access personal google cloud services
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
# Installs
!pip install ipdb

In [ ]:
# Imports

# Data manipulation
import pandas as pd

# Regular expressions
import re

# Grab data from database
from google.cloud import bigquery

# Debugging
import ipdb

In [ ]:
# Turning on automatic debugger
%pdb on

Automatic pdb calling has been turned ON


# LOADING DATA (BigQuery queries)

In [ ]:
# Client connect to bigquery project
client = bigquery.Client('nfl-data-430702')

## Season 2023 Week 1

In [ ]:
# Grabbing all plays from Super Bowl 2023
week1_2023_plays_query = """
                         SELECT *
                         FROM `nfl-data-430702.NFL_Scores.NFL-Plays-Week1_2023`
                         """

# Running psuedo query, and returns the amount of bytes it will take to run query
dry_run_config = bigquery.QueryJobConfig(dry_run=True)
dry_run_query = client.query(week1_2023_plays_query, job_config=dry_run_config)
print("This query will process {} bytes.".format(dry_run_query.total_bytes_processed))

# Running query (Being mindful of the amount of data being grabbed)
# Will grab a maximum of a Gigabyte
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**9)
safe_config_query = client.query(week1_2023_plays_query, job_config=safe_config)

This query will process 570194 bytes.


In [ ]:
# Putting data attained from query into a dataframe
week1_2023_plays = safe_config_query.to_dataframe()

In [ ]:
week1_2023_plays.head()

,Season,Week,Day,Date,AwayTeam,HomeTeam,Quarter,DriveNumber,TeamWithPossession,IsScoringDrive,PlayNumberInDrive,IsScoringPlay,PlayOutcome,PlayDescription,PlayStart
0,2023,Week 1,MON,09/11,Bills,Jets,1ST QUARTER,1,BUF,0,1,0,Kickoff,G.Zuerlein kicks 65 yards from NYJ 35 to end z...,Kickoff from NYJ 35
1,2023,Week 1,MON,09/11,Bills,Jets,1ST QUARTER,1,BUF,0,2,0,7 Yard Pass,(15:00) (Shotgun) J.Allen pass short right to ...,1st & 10 at BUF 25
2,2023,Week 1,MON,09/11,Bills,Jets,1ST QUARTER,1,BUF,0,3,0,5 Yard Pass,"(14:34) (No Huddle, Shotgun) J.Allen pass shor...",2nd & 3 at BUF 32
3,2023,Week 1,MON,09/11,Bills,Jets,1ST QUARTER,1,BUF,0,4,0,3 Yard Run,(14:01) J.Cook up the middle to BUF 40 for 3 y...,1st & 10 at BUF 37
4,2023,Week 1,MON,09/11,Bills,Jets,1ST QUARTER,1,BUF,0,5,0,2 Yard Run,(13:24) (Shotgun) J.Cook up the middle to BUF ...,2nd & 7 at BUF 40


In [ ]:
# Observation of the amount of data being worked on
week1_2023_plays.shape

(2600, 15)

# CATEGORIZE PLAYS
- The goal here is to parse out the different values for 'PlayOutcome'
  - separate pass / run / kickoff / etc.

## PARSING


In [ ]:
# Maybe try to fuzzywuzzy this in the future?

# All play outcomes from the game
# - From here we can categorize and clean plays accordingly
week1_2023_plays['PlayOutcome'].unique()

array(['Kickoff', '7 Yard Pass', '5 Yard Pass', '3 Yard Run',
       '2 Yard Run', 'Pass Incomplete', 'Punt', '-5 Yard Penalty',
       '5 Yard Run', '1 Yard Pass', '14 Yard Run', '3 Yard Pass',
       '8 Yard Run', '6 Yard Pass', '15 Yard Pass', '-9 Yard Sack',
       '4 Yard Pass', '13 Yard Pass', 'Field Goal', '-2 Yard Sack',
       'Interception', '-5 Yard Run', '18 Yard Pass', '8 Yard Pass',
       '6 Yard Run', '12 Yard Run', '-1 Yard Run', '26 Yard Pass',
       'Touchdown Bills', 'Extra Point Good', '13 Yard Run',
       '-3 Yard Sack', '7 Yard Run', '9 Yard Pass', '4 Yard Run',
       'Fumble', '-10 Yard Penalty', '10 Yard Pass', '26 Yard Run',
       '5 Yard Penalty', '-10 Yard Sack', '22 Yard Pass', '-4 Yard Run',
       '-12 Yard Sack', '83 Yard Run', '1 Yard Run', '2 Yard Pass',
       '10 Yard Run', 'Run for No Gain', '12 Yard Pass', '20 Yard Pass',
       '9 Yard Run', '-2 Yard Pass', 'Sack', '24 Yard Pass',
       '14 Yard Pass', 'Touchdown Jets', '-3 Yard Run', '-2 Yar

In [ ]:
# There are more types of plays that I have not made yet for Week 1.

# Looking at all unique play outcomes and categorizing them.
# - This type of approach does not feel very flexable because a play outcome can
#   arise that has not been seen yet.
# - There may be more in the future when working on a full season, let alone all seasons and future games
df_2023_pass_week1 = week1_2023_plays[week1_2023_plays['PlayOutcome'].str.contains('Pass')]
df_2023_run_week1 = week1_2023_plays[week1_2023_plays['PlayOutcome'].str.contains('Run')]

# df_2023_punt_week1 = week1_2023_plays[week1_2023_plays['PlayOutcome'].str.contains('Punt')]
# df_2023_sack_week1 = week1_2023_plays[week1_2023_plays['PlayOutcome'].str.contains('Sack')]
# df_2023_kickoff_week1 = week1_2023_plays[week1_2023_plays['PlayOutcome'].str.contains('Kickoff')]
# df_2023_fumble_week1 = week1_2023_plays[week1_2023_plays['PlayOutcome'].str.contains('Fumble')]
# df_2023_interception_week1 = week1_2023_plays[week1_2023_plays['PlayOutcome'].str.contains('Interception')]
# df_2023_penalty_week1 = week1_2023_plays[week1_2023_plays['PlayOutcome'].str.contains('Penalty')]
# df_2023_fieldgoal_week1 = week1_2023_plays[week1_2023_plays['PlayOutcome'].str.contains('Field Goal')]
# df_2023_touchdown_week1 = week1_2023_plays[week1_2023_plays['PlayOutcome'].str.contains('Touchdown')]
# df_2023_extrapoint_week1 = week1_2023_plays[week1_2023_plays['PlayOutcome'].str.contains('Extra Point')]

# plays_list = [df_2023_pass_sb,
#               df_2023_run_sb,
#               df_2023_punt_sb,
#               df_2023_sack_sb,
#               df_2023_kickoff_sb,
#               df_2023_fumble_sb,
#               df_2023_interception_sb,
#               df_2023_penalty_sb,
#               df_2023_fieldgoal_sb,
#               df_2023_touchdown_sb,
#               df_2023_extrapoint_sb]

## SANITY CHECK (All Plays Accounted for)
- NOT COMPLETE
  - Still need to grab other play types

## HELPER METHODS

In [ ]:
# PURPOSE:
# - Quick look at a section of plays
#   - Ideally the plays that the user wants to break down and clean.
# INPUT PARAMETERS:
# df_all_plays      - DataFrame - The original dataframe where the desired plays to view came from
# df_section_plays  - DataFrame - A section of the original dataframe the user wants to view
# RETURN:
# - Printing to the console:
#   1. index of play
#   2. 'PlayDescription' feature of play
#   3. 'PlayOutcome' feature of play
def print_plays(df_all_plays, df_section_plays):
  for idx, value in df_section_plays['PlayOutcome'].items():
    print("index:" + str(idx))
    play = df_all_plays['PlayDescription'].iloc[idx]
    print(play)
    print(value)
    print()

# Fumbled plays (Pass & Run)
- Only looking for fumbled plays

In [1]:
# 1.2 ATTEMPT TO HANDLE FUMBLES

# The goal for this (soon to be) method is to extract all data from a fumble play
# that is uncommon to generic play types such as pass or run plays. All data that
# is common will be pushed and cleaned with the generic play type procedures.

# Regular expression used to grab QB only fumbles.
# Example - "(14:21) J.Love to CHI 44 for -3 yards"
# NOTE:
# - There are other plays that follow this format.
#   So far I have seen:
#   1. E.Speed ran ob at IND 49 for no gain
#      - What looks like to be an ordinary run play
qb_fumble = "[A-Za-z]+\.[A-Za-z]+-?[A-Za-z]* to [A-Z]+ [0-9]+ for -?[0-9]+ yards$"

# Regular expression used for players who recovered the fumbled ball.
# Example: NYG-P.Campbell
fumble_recoverer = "[A-Z]+-[A-Z]+\.[A-Za-z]+-?[A-Za-z]*"

for idx, value in df_2023_pass_week1['PlayOutcome'].items():
  play = week1_2023_plays['PlayDescription'].iloc[idx]

  # Grabbing all plays containing the string "FUMBLES" -> splitting the play by sentences.
  # - Each sentence represents a single moment of the play and can be broken down easier.
  #   Example: (14:21) J.Love to CHI 44 for -3 yards.
  #            FUMBLES, and recovers at CHI 46.
  #            J.Love pass deep left to L.Musgrave to CHI 4 for 37 yards (T.Stevenson) [D.Walker].
  if play.find('FUMBLES') != -1:
    # All moments within a set list.
    # - Will not be changed
    fumble_play_elements = play.split(". ")
    # All moments within the list that has to do with the fumble only.
    # - Goal is to keep them in the same order as when they appeared originally.
    fumble_details = [None] * len(fumble_play_elements)
    # All moments within the play that will be pushed to generic play type breakdown procedure
    # - generic play type procedures like run/pass play
    main_play = []

    # Can be seen as a switch that, when true, will automatically add the next
    # moment to 'fumble_details'
    # - Currently only used for recovered fumble action that does not count towards
    #   offensive player stats such as passing/reception yards or yards rushed.
    automatic_add = False

    for i in fumble_play_elements:

      # All moments within play will be added to 'main_play' and filtered out from there.
      if automatic_add:
        fumble_details.pop(fumble_play_elements.index(i))
        fumble_details.insert(fumble_play_elements.index(i), i)
        automatic_add = False
        continue
      else:
        main_play.append(i)

      # ~ QB only fumbles ~
      passer = re.findall(qb_fumble, i)
      if len(passer) == 1:
        # - There are 2 possible sentences that will pass to here.
        #     1. QB only fumble (e.g. (14:21) J.Love to CHI 44 for -3 yards) - normally the 1st sentence
        #     2. Fumble recovery action (e.g. P.Campbell to NYG 33 for -2 yards) - follows FUMBLES sentence
        #   - What separates them is where the 'FUMBLES' moment occurs in relation
        #     - If it is directly before then it is (2) else (1) <- (1) is what we want.
        if fumble_play_elements.index(i) > 0 and fumble_play_elements[fumble_play_elements.index(i)-1].find('FUMBLES') != -1:
          continue
        else:
          # Take out QB only fumble and add to fumble_details
          main_play.pop(main_play.index(i))
          fumble_details.pop(fumble_play_elements.index(i))
          fumble_details.insert(fumble_play_elements.index(i), i)

      # ~ Fumble and recovery ~
      # Example: "FUMBLES (M.Bell), recovered by NYG-P.Campbell at NYG 35"
      if i.find('FUMBLES') != -1:
        # If the player that recovered the ball in the fumbles sentence is within the
        # sentence after, then the sentence after will automatically be added into "fumble_details".
        # Example of sentence after: P.Campbell to NYG 33 for -2 yards
        recoverer = re.findall(fumble_recoverer, i)
        if len(recoverer) > 0:
          player_who_recovered_ball = recoverer[0][recoverer[0].find("-") + 1:] # NYG-P.Campbell -> P.Campbell
          try:
            if fumble_play_elements[fumble_play_elements.index(i)+1].find(player_who_recovered_ball) != -1:
              # The next
              automatic_add = True
          except IndexError:
            pass
        main_play.pop(main_play.index(i))
        fumble_details.pop(fumble_play_elements.index(i))
        fumble_details.insert(fumble_play_elements.index(i), i)

      # Why is this here when all play types can be 'REVERSED'?
      # - A fumbled play that has been reversed can result to another fumbled play.
      #   - Since this grabs all fumbled plays including the ones that are reversed,
      #     I think it makes sense to keep it in here. Where it can grab all details
      #     relating to the fumble, even though all actions taken before the "reverse"
      #     are not commonly recorded in player stats.
      if i.find('REVERSED') != -1:
        for j in main_play:
          fumble_details.pop(fumble_play_elements.index(j))
          fumble_details.insert(fumble_play_elements.index(j), j)
        main_play.clear()

    print(fumble_details)
    print(main_play)
    print(value)
    print()

    # Now it's time to prep 'main_play' to be able to be cleaned by the
    # generic play type procedures.
    print(". ".join(main_play))

NameError: name 'df_2023_pass_week1' is not defined

In [ ]:
for idx, value in df_2023_pass_week1['PlayOutcome'].items():
  play = week1_2023_plays['PlayDescription'].iloc[idx]
  if play.find('FUMBLES') != -1:
    print("index:" + str(idx))
    fumble_play_elements = play.split(". ")
    for i in fumble_play_elements:
      print(i)
    # print(play)
    print(value)
    print()

index:213
(14:21) J.Love to CHI 44 for -3 yards
FUMBLES, and recovers at CHI 46
J.Love pass deep left to L.Musgrave to CHI 4 for 37 yards (T.Stevenson) [D.Walker].
37 Yard Pass

index:423
(14:15) T.Lawrence pass short right to C.Ridley to JAX 47 for 14 yards (R.Thomas, E.Speed)
FUMBLES (E.Speed), RECOVERED by IND-E.Speed at IND 49
E.Speed ran ob at IND 49 for no gain
The Replay Official reviewed the ball was inbounds ruling, and the play was REVERSED
T.Lawrence pass short right to C.Ridley to JAX 47 for 14 yards (R.Thomas, E.Speed)
FUMBLES (E.Speed), ball out of bounds at IND 49
IND-K.Moore was injured during the play
IND-D.Flowers was injured during the play.
14 Yard Pass

index:872
(11:26) (Shotgun) D.Prescott pass short right to T.Pollard to NYG 12 for 7 yards (B.Okereke)
FUMBLES (B.Okereke), recovered by DAL-T.Biadasz at NYG 4.
7 Yard Pass

index:961
(4:45) (Shotgun) D.Jones pass short left to M.Breida to NYG 43 for 5 yards (M.Bell)
FUMBLES (M.Bell), recovered by NYG-P.Campbell at 